In [1]:
!pip install -q protobuf==3.20.3
!pip install -q -U transformers datasets peft accelerate evaluate sacrebleu scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [2]:
import os
import gc
import torch
import numpy as np
import evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType

# =========================
# 1. CONFIG & SETUP
# =========================
# Xóa bộ nhớ cũ
gc.collect()
torch.cuda.empty_cache()

MODEL_ID = "facebook/nllb-200-distilled-600M"
OUTPUT_DIR = "./nllb_medical_finetune"

# Đường dẫn file (Sửa lại nếu cần)
TRAIN_VI_PATH = "/kaggle/input/vlsp-data/MedicalDataset_VLSP/train.vi.txt"
TRAIN_EN_PATH = "/kaggle/input/vlsp-data/MedicalDataset_VLSP/train.en.txt"

# Cấu hình
MAX_LENGTH = 128
BATCH_SIZE = 16   
MAX_SAMPLES = 250000 

# Load metric BLEU
metric = evaluate.load("sacrebleu")

# =========================
# 2. LOAD DATA & PREPROCESS
# =========================
# NLLB cần src_lang (nguồn) và tgt_lang (đích)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, src_lang="vie_Latn", tgt_lang="eng_Latn")

def load_data():
    if os.path.exists(TRAIN_VI_PATH) and os.path.exists(TRAIN_EN_PATH):
        print("📂 Đang đọc file dữ liệu...")
        data = []
        with open(TRAIN_VI_PATH, encoding="utf-8") as f_vi, \
             open(TRAIN_EN_PATH, encoding="utf-8") as f_en:
            for i, (vi, en) in enumerate(zip(f_vi, f_en)):
                if i >= MAX_SAMPLES: break
                if vi.strip() and en.strip():
                    data.append({"vi": vi.strip(), "en": en.strip()})
        return Dataset.from_list(data)
    else:
        # Dữ liệu giả lập nếu không tìm thấy file
        print("⚠️ Không thấy file, tạo dữ liệu giả lập...")
        dummy = [{"vi": "Bệnh nhân đau đầu.", "en": "The patient has a headache."}] * 100
        return Dataset.from_list(dummy)

raw_dataset = load_data()

# Chia tập Train (90%) và Test (10%) để chấm điểm BLEU
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split_dataset["train"]
test_ds = split_dataset["test"]

print(f"✅ Train size: {len(train_ds)} | Test size: {len(test_ds)}")

def preprocess_function(examples):
    inputs = [ex for ex in examples["vi"]]
    targets = [ex for ex in examples["en"]]
    model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_LENGTH, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map dữ liệu
tokenized_train = train_ds.map(preprocess_function, batched=True, remove_columns=["vi", "en"])
tokenized_test = test_ds.map(preprocess_function, batched=True, remove_columns=["vi", "en"])

# =========================
# 3. MODEL & LORA
# =========================
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, device_map="auto")

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"] # Target full
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# =========================
# 4. HÀM TÍNH BLEU (QUAN TRỌNG)
# =========================
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode kết quả dự đoán
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Thay -100 bằng pad_token_id để decode được nhãn
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Chuẩn hóa văn bản nhẹ
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    # Tính BLEU
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

# =========================
# 5. TRAINING ARGS
# =========================
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=100,
    fp16=True, # Tăng tốc
    save_strategy="no",
    eval_strategy="no", # Tắt eval trong lúc train cho nhanh, sẽ eval sau
    predict_with_generate=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# =========================
# 6. TRAIN & EVALUATE
# =========================
print("\n🚀 BẮT ĐẦU TRAINING...")
trainer.train()



2025-12-23 09:01:39.678620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766480500.056230      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766480500.168364      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

trainable params: 8,650,752 || all params: 623,724,544 || trainable%: 1.3870

🚀 BẮT ĐẦU TRAINING...


/tmp/ipykernel_47/1855590104.py:140: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
100,1.918200
200,1.862800
300,1.796600
400,1.750300
500,1.751100
600,1.733800
700,1.722000
800,1.693100
900,1.705100
1000,1.654900


TrainOutput(global_step=7032, training_loss=1.5993203015593267, metrics={'train_runtime': 6010.2299, 'train_samples_per_second': 37.436, 'train_steps_per_second': 1.17, 'total_flos': 4.41436787884032e+16, 'train_loss': 1.5993203015593267, 'epoch': 1.0})

In [8]:
!pip install -q evaluate jiwer nltk
import nltk
import pandas as pd
import evaluate

# Tải dữ liệu cần thiết cho METEOR
nltk.download("wordnet")
nltk.download("punkt")
nltk.download('omw-1.4') # Quan trọng cho wordnet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.6 MB/s eta 0:00:0000:0100:01


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [10]:
# Load các metrics
bleu_metric = evaluate.load("sacrebleu")
meteor_metric = evaluate.load("meteor")
ter_metric = evaluate.load("ter")

def generate_comprehensive_report(dataset, max_samples=100):
    print(f"🚀 Đang chạy đánh giá trên {max_samples} mẫu...")
    
    # Lấy mẫu
    subset = dataset.select(range(min(len(dataset), max_samples)))
    inputs = subset["input_ids"]
    labels = subset["labels"]
    
    model.eval()
    predictions = []
    
    batch_size = 8
    total_batches = (len(inputs) + batch_size - 1) // batch_size
    
    # Thanh tiến trình đơn giản
    print(f"🔄 Đang xử lý {total_batches} batches...")

    for i in range(total_batches):
        # 1. Lấy batch thô (các list độ dài khác nhau)
        batch_input_ids = inputs[i*batch_size : (i+1)*batch_size]
        
        # 2. FIX LỖI Ở ĐÂY: Dùng tokenizer để Pad cho bằng nhau
        # Chuyển list of list thành list of dict để tokenizer hiểu
        batch_data = [{"input_ids": ids, "attention_mask": [1]*len(ids)} for ids in batch_input_ids]
        
        # Pad tự động thành tensor vuông vức
        padded_inputs = tokenizer.pad(
            batch_data,
            padding=True,
            return_tensors="pt"
        ).to("cuda")
        
        input_ids = padded_inputs["input_ids"]
        attention_mask = padded_inputs["attention_mask"]
        
        # 3. Dự đoán
        with torch.no_grad():
            generated_tokens = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask, # Thêm mask cho chuẩn
                forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"),
                max_length=128
            )
        
        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        predictions.extend(decoded_preds)

    # Xử lý nhãn (Ground Truth)
    clean_labels = []
    for lbl in labels:
        lbl_clean = [l if l != -100 else tokenizer.pad_token_id for l in lbl]
        clean_labels.append(tokenizer.decode(lbl_clean, skip_special_tokens=True))
    
    # Decode Input để lưu file
    # (Phải pad input toàn cục để decode hàng loạt, hoặc decode từng cái)
    # Cách nhanh: decode từng cái trong loop input gốc
    input_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in inputs]

    print("📊 Đang tính toán các chỉ số...")
    
    bleu_score = bleu_metric.compute(predictions=predictions, references=clean_labels)
    meteor_score = meteor_metric.compute(predictions=predictions, references=clean_labels)
    ter_score = ter_metric.compute(predictions=predictions, references=clean_labels)
    
    print("\n" + "="*40)
    print(f"🏆 KẾT QUẢ ĐÁNH GIÁ (Trên {max_samples} câu):")
    print(f"🔹 BLEU:   {bleu_score['score']:.2f}")
    print(f"🔹 METEOR: {meteor_score['meteor']:.4f}")
    print(f"🔹 TER:    {ter_score['score']:.2f}")
    print("="*40)

    print("\n📝 Đang tạo file báo cáo lỗi (error_analysis.csv)...")
    df = pd.DataFrame({
        "Source (VI)": input_texts,
        "Target (EN)": clean_labels,
        "Model Output": predictions
    })
    df.to_csv("error_analysis.csv", index=False, encoding="utf-8-sig")
    print("✅ Xong!")
    
    return df

# Chạy lại
report_df = generate_comprehensive_report(tokenized_test, max_samples=100)
print(report_df.head())

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
You're using a NllbTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


🚀 Đang chạy đánh giá trên 100 mẫu...
🔄 Đang xử lý 13 batches...
📊 Đang tính toán các chỉ số...

🏆 KẾT QUẢ ĐÁNH GIÁ (Trên 100 câu):
🔹 BLEU:   33.60
🔹 METEOR: 0.6199
🔹 TER:    62.31

📝 Đang tạo file báo cáo lỗi (error_analysis.csv)...
✅ Xong!
                                         Source (VI)  \
0  Phương pháp: Nghiên cứu theo phương pháp mô tả...   
1  Kết quả: Không có tử vong sau mổ và biến chứng...   
2  Kết quả: Mô hình lí thuyết trên điện toán cắt ...   
3                 BMI nên được tính toán và ghi lại.   
4  Một thử nghiệm lâm sàng không làm mù gần đây v...   

                                         Target (EN)  \
0  Material and method: Cross sectional and descr...   
1  Results: No operated mortality and 13.2% minim...   
2  Results: In the theoretical arthroplasty model...   
3  Body mass index (BMI) should be calculated and...   
4  A recent unblinded clinical trial of dexametha...   

                                        Model Output  
0        Methods: Cross-sectio